In [ ]:
from tmlt.analytics.query_builder import QueryBuilder
from tmlt.analytics.privacy_budget import PureDPBudget
from tmlt.analytics.session import Session
from tmlt.analytics.keyset import KeySet

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.getOrCreate()
private_data = spark.createDataFrame(pd.DataFrame([["0", 1, 0], ["1", 0, 1]], columns=["A", "B", "X"]))
public_data = spark.createDataFrame(pd.DataFrame([["0", 1], ["1", 0]], columns=["A", "Y"]))
total_budget = 10

### Create Session

In [ ]:
sess = Session.from_dataframe(
    privacy_budget=PureDPBudget(total_budget),
    source_id="my_private_data",
    dataframe=private_data,
)

In [ ]:
sess.private_sources

In [ ]:
sess.add_public_dataframe("my_public_data", public_data)

In [ ]:
sess.public_sources

### Create View

In [ ]:
join_query = QueryBuilder("my_private_data").join_public("my_public_data")
sess.create_view(join_query, "joined_data", cache=True)

In [ ]:
sess.private_sources

In [ ]:
sess.get_schema("joined_data")

In [ ]:
count_query = QueryBuilder("joined_data").groupby(KeySet.from_dict({"Y": [0, 1]})).count()
sess.evaluate(count_query, privacy_budget=PureDPBudget(total_budget/5)).show()

In [ ]:
sess.remaining_privacy_budget

### Partition the Session

In [ ]:
new_sessions = sess.partition_and_create(
    "joined_data", 
    privacy_budget=PureDPBudget(3),
    column="Y", 
    splits={"partition_0": 0, "partition_1": 1}
)

In [ ]:
sess.remaining_privacy_budget

In [ ]:
new_sessions["partition_0"].remaining_privacy_budget

In [ ]:
new_sessions["partition_0"].private_sources

In [ ]:
new_sessions["partition_0"].get_schema("partition_0")

In [ ]:
new_sessions["partition_1"].remaining_privacy_budget

In [ ]:
new_sessions["partition_1"].private_sources